In [1]:
import spacy
from keras.preprocessing.text import Tokenizer
from keras.utils import to_categorical
from keras.models import Sequential
from keras.layers import Dense,LSTM,Embedding
from pickle import dump,load
from keras.preprocessing.sequence import pad_sequences
import numpy as np

In [2]:
nlp = spacy.load('en', disable=['parser', 'tagger', 'ner'])
nlp.max_length = 1198623

In [3]:
def read_file(filepath):
    with open(filepath) as f:
        str_text = f.read()
        
    return str_text

In [4]:
def separate_punc(doc_text):
    return [token.text.lower() for token in nlp(doc_text) if token.text not in '\n\n \n\n\n!"-#$%&()--.*+,-/:;<=>?@[\\]^_`{|}~\t\n ']

In [5]:
d = read_file('moby_dick_four_chapters.txt')

In [6]:
tokens = separate_punc(d)

In [7]:
# 25 words ---> network predict #26
train_len = 25 + 1

text_sequences = []

for i in range(train_len, len(tokens)):
    seq = tokens[i-train_len: i]
    text_sequences.append(seq)
    

In [8]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts(text_sequences)

In [9]:
sequences = tokenizer.texts_to_sequences(text_sequences)

In [10]:
for i in sequences[0]:
    print(f"{i}: {tokenizer.index_word[i]}")

956: call
14: me
263: ishmael
51: some
261: years
408: ago
87: never
219: mind
129: how
111: long
954: precisely
260: having
50: little
43: or
38: no
315: money
7: in
23: my
546: purse
3: and
150: nothing
259: particular
6: to
2712: interest
14: me
24: on


In [11]:
vocabulary_size = len(tokenizer.word_counts)
vocabulary_size

2717

In [12]:
sequences = np.array(sequences)
sequences

array([[ 956,   14,  263, ..., 2712,   14,   24],
       [  14,  263,   51, ...,   14,   24,  957],
       [ 263,   51,  261, ...,   24,  957,    5],
       ...,
       [ 952,   12,  166, ...,  262,   53,    2],
       [  12,  166, 2711, ...,   53,    2, 2717],
       [ 166, 2711,    3, ...,    2, 2717,   26]])

In [13]:
X = sequences[:, :-1]
y = sequences[:, -1]

In [14]:
y = to_categorical(y, num_classes=vocabulary_size+1)

In [15]:
y.shape

(11312, 2718)

In [16]:
seq_len = X.shape[1]

In [17]:
def create_model(vocabulary_size, seq_len):
    model = Sequential()
    model.add(Embedding(vocabulary_size, seq_len, input_length=seq_len))
    model.add(LSTM(2*seq_len, return_sequences=True))
    model.add(LSTM(2*seq_len))
    model.add(Dense(50, activation='relu'))
    model.add(Dense(vocabulary_size, activation='softmax'))
    
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    model.summary()
    return model

In [18]:
model = create_model(vocabulary_size+1, seq_len)

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 25, 25)            67950     
_________________________________________________________________
lstm (LSTM)                  (None, 25, 50)            15200     
_________________________________________________________________
lstm_1 (LSTM)                (None, 50)                20200     
_________________________________________________________________
dense (Dense)                (None, 50)                2550      
_________________________________________________________________
dense_1 (Dense)              (None, 2718)              138618    
Total params: 244,518
Trainable params: 244,518
Non-trainable params: 0
_________________________________________________________________


In [19]:
model.fit(X, y ,batch_size=128, epochs=300, verbose=1)

Epoch 1/300
89/89 [==============================] - 2s 23ms/step - loss: 6.9799 - accuracy: 0.0445
Epoch 2/300
89/89 [==============================] - 2s 22ms/step - loss: 6.3741 - accuracy: 0.0529
Epoch 3/300
89/89 [==============================] - 2s 22ms/step - loss: 6.3468 - accuracy: 0.0529
Epoch 4/300
89/89 [==============================] - 2s 22ms/step - loss: 6.3202 - accuracy: 0.0529
Epoch 5/300
89/89 [==============================] - 2s 22ms/step - loss: 6.2130 - accuracy: 0.0529
Epoch 6/300
89/89 [==============================] - 2s 22ms/step - loss: 6.1329 - accuracy: 0.0529
Epoch 7/300
89/89 [==============================] - 2s 22ms/step - loss: 6.0592 - accuracy: 0.0533
Epoch 8/300
89/89 [==============================] - 2s 23ms/step - loss: 5.9607 - accuracy: 0.0545
Epoch 9/300
89/89 [==============================] - 3s 30ms/step - loss: 5.8912 - accuracy: 0.0574
Epoch 10/300
89/89 [==============================] - 3s 31ms/step - loss: 5.8330 - accuracy: 0.0650

In [20]:
def generate_text(model, tokenizer, seq_len, seed_text, num_gen_words):
    output_text = []
    
    input_text = seed_text
    
    for i in range(num_gen_words):
        encoded_text = tokenizer.texts_to_sequences([input_text])[0]
        
        pad_encoded = pad_sequences([encoded_text], maxlen=seq_len, truncating='pre')
        
        pred_word_ind = model.predict_classes(pad_encoded, verbose=0)[0]
        
        pred_word = tokenizer.index_word[pred_word_ind]
        
        input_text += ' ' +  pred_word
        
        output_text.append(pred_word)
    
    return ' '.join(output_text)

In [21]:
import random
random.seed(101)
random_pick = random.randint(0, len(text_sequences))

In [22]:
random_seed_text = text_sequences[random_pick]

In [23]:
seed_text = " ".join(random_seed_text)
seed_text

"thought i to myself the man 's a human being just as i am he has just as much reason to fear me as i have"

In [24]:
generate_text(model, tokenizer, seq_len, seed_text, num_gen_words=25)

Instructions for updating:
Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).


'been to tell that the morning for it was it was not the devil himself he passed it and ee obey and demand to the'